# AI Stock Management - MySQL Database Setup & Analysis

## Complete Database Setup for Low Stock Alerts System

This notebook provides a corrected implementation of the AI Stock Management database schema, addressing MySQL 8.0 compatibility issues and providing comprehensive analysis tools.

### Issues Identified & Solutions:
1. **MySQL Syntax Errors**: `IF NOT EXISTS` not supported in `ALTER TABLE` and `CREATE INDEX`
2. **Missing Tables**: Need to create `products` table before dependent tables
3. **Foreign Key Dependencies**: Correct order of table creation required

### What This Notebook Covers:
- ✅ **Corrected SQL Scripts** for MySQL 8.0 compatibility
- ✅ **Step-by-Step Database Setup** with proper table creation order
- ✅ **Data Analysis & Visualization** of stock alerts and purchase orders
- ✅ **Performance Monitoring** and optimization recommendations

In [ ]:
# Database Connection Setup
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import json

# Configure visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Database connection configuration
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',  # Add your MySQL password here
    'database': 'DemoDemo',  # Your database name
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}

def get_connection():
    """Create and return database connection"""
    try:
        connection = pymysql.connect(**DB_CONFIG)
        print("✅ Database connection successful!")
        return connection
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return None

# Test connection
conn = get_connection()

## 1. Corrected Database Schema Setup

### Step 1: Create Tables in Correct Order

The main issue was trying to create dependent tables before their parent tables existed. Here's the corrected order:

In [ ]:
# Step 1: Create Suppliers Table (No dependencies)
def create_suppliers_table(connection):
    """Create suppliers table"""
    sql = """
    CREATE TABLE IF NOT EXISTS suppliers (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        email VARCHAR(255),
        phone VARCHAR(20),
        address TEXT,
        contact_person VARCHAR(255),
        website VARCHAR(255),
        tax_number VARCHAR(50),
        payment_terms INT DEFAULT 30,
        is_active BOOLEAN DEFAULT TRUE,
        notes TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        INDEX idx_supplier_name (name),
        INDEX idx_supplier_active (is_active)
    );
    """
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            connection.commit()
            print("✅ Suppliers table created successfully")
    except Exception as e:
        print(f"❌ Error creating suppliers table: {e}")

# Step 2: Create Products Table (Depends on suppliers)
def create_products_table(connection):
    """Create products table with all alert management columns"""
    sql = """
    CREATE TABLE IF NOT EXISTS products (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        category VARCHAR(100),
        price DECIMAL(10,2) NOT NULL,
        current_stock INT DEFAULT 0,
        low_stock_threshold INT DEFAULT 10,
        low_stock_alert_time TIMESTAMP NULL,
        alert_status ENUM('active', 'ignored', 'resolved') DEFAULT 'active',
        qr_code VARCHAR(100) UNIQUE,
        unit VARCHAR(20) DEFAULT 'piece',
        supplier_id INT,
        alert_priority ENUM('low', 'medium', 'high') DEFAULT 'medium',
        reorder_point INT DEFAULT 0,
        max_stock_level INT DEFAULT 1000,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        FOREIGN KEY (supplier_id) REFERENCES suppliers(id) ON DELETE SET NULL,
        INDEX idx_products_stock (current_stock, low_stock_threshold),
        INDEX idx_products_category (category),
        INDEX idx_products_supplier (supplier_id)
    );
    """
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            connection.commit()
            print("✅ Products table created successfully")
    except Exception as e:
        print(f"❌ Error creating products table: {e}")

# Execute table creation
if conn:
    create_suppliers_table(conn)
    create_products_table(conn)

In [ ]:
# Step 3: Create dependent tables
def create_dependent_tables(connection):
    """Create all tables that depend on products and suppliers"""
    
    # Purchase Orders Table
    purchase_orders_sql = """
    CREATE TABLE IF NOT EXISTS purchase_orders (
        id INT AUTO_INCREMENT PRIMARY KEY,
        po_number VARCHAR(50) UNIQUE NOT NULL,
        product_id INT NOT NULL,
        supplier_id INT,
        quantity INT NOT NULL,
        unit_price DECIMAL(10,2) NOT NULL,
        total_amount DECIMAL(10,2) NOT NULL,
        status ENUM('pending', 'sent', 'approved', 'delivered', 'completed', 'cancelled') DEFAULT 'pending',
        priority ENUM('low', 'medium', 'high', 'urgent') DEFAULT 'medium',
        notes TEXT,
        sent_method ENUM('email', 'whatsapp', 'phone', 'fax') NULL,
        sent_to VARCHAR(255) NULL,
        sent_at TIMESTAMP NULL,
        expected_delivery_date DATE NULL,
        actual_delivery_date DATE NULL,
        delivery_address TEXT,
        terms_and_conditions TEXT,
        created_by VARCHAR(255) DEFAULT 'System',
        approved_by VARCHAR(255) NULL,
        approved_at TIMESTAMP NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE CASCADE,
        FOREIGN KEY (supplier_id) REFERENCES suppliers(id) ON DELETE SET NULL,
        INDEX idx_po_status (status),
        INDEX idx_po_date (created_at),
        INDEX idx_po_supplier (supplier_id),
        INDEX idx_po_product (product_id)
    );
    """
    
    # Ignored Alerts Table
    ignored_alerts_sql = """
    CREATE TABLE IF NOT EXISTS ignored_alerts (
        id INT AUTO_INCREMENT PRIMARY KEY,
        product_id INT NOT NULL,
        original_alert_time TIMESTAMP NOT NULL,
        reason TEXT,
        ignored_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        ignored_by VARCHAR(255) DEFAULT 'Admin',
        stock_level_when_ignored INT,
        can_reactivate BOOLEAN DEFAULT TRUE,
        reactivate_after TIMESTAMP NULL,
        FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE CASCADE,
        INDEX idx_ignored_product (product_id),
        INDEX idx_ignored_date (ignored_at)
    );
    """
    
    # Alert History Table
    alert_history_sql = """
    CREATE TABLE IF NOT EXISTS alert_history (
        id INT AUTO_INCREMENT PRIMARY KEY,
        product_id INT NOT NULL,
        action ENUM('created', 'ignored', 'resolved', 'reactivated', 'po_generated') NOT NULL,
        stock_level INT,
        threshold_level INT,
        performed_by VARCHAR(255) DEFAULT 'System',
        notes TEXT,
        metadata JSON,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE CASCADE,
        INDEX idx_history_product (product_id),
        INDEX idx_history_action (action),
        INDEX idx_history_date (created_at)
    );
    """
    
    tables = [
        ("purchase_orders", purchase_orders_sql),
        ("ignored_alerts", ignored_alerts_sql),
        ("alert_history", alert_history_sql)
    ]
    
    for table_name, sql in tables:
        try:
            with connection.cursor() as cursor:
                cursor.execute(sql)
                connection.commit()
                print(f"✅ {table_name} table created successfully")
        except Exception as e:
            print(f"❌ Error creating {table_name} table: {e}")

# Execute dependent tables creation
if conn:
    create_dependent_tables(conn)

In [ ]:
# Step 4: Insert Sample Data
def insert_sample_data(connection):
    """Insert comprehensive sample data for testing"""
    
    # Insert Suppliers
    suppliers_data = [
        (1, 'Coffee World Ltd', 'orders@coffeeworld.com', '+1-555-0101', 'John Smith', '123 Coffee St, Seattle, WA 98101', 'www.coffeeworld.com', 30),
        (2, 'Herbal Supplies Co', 'supply@herbal.com', '+1-555-0102', 'Sarah Johnson', '456 Green Ave, Portland, OR 97201', 'www.herbalsupplies.com', 15),
        (3, 'Tech Solutions Inc', 'orders@techsol.com', '+1-555-0103', 'Mike Chen', '789 Tech Blvd, San Jose, CA 95101', 'www.techsolutions.com', 45),
        (4, 'Office Depot Pro', 'bulk@officedepot.com', '+1-555-0104', 'Lisa Brown', '321 Business Rd, Dallas, TX 75201', 'www.officedepot.com', 30),
        (5, 'General Supplies Hub', 'info@generalsupplies.com', '+1-555-0105', 'David Wilson', '654 Supply Lane, Chicago, IL 60601', 'www.generalsupplies.com', 20),
        (6, 'Fresh Foods Wholesale', 'wholesale@freshfoods.com', '+1-555-0106', 'Emma Davis', '987 Fresh Market St, Miami, FL 33101', 'www.freshfoods.com', 7),
        (7, 'Electronic Components Ltd', 'sales@electronics.com', '+1-555-0107', 'James Miller', '147 Circuit Ave, Austin, TX 78701', 'www.electronics.com', 60)
    ]
    
    suppliers_sql = """
    INSERT IGNORE INTO suppliers (id, name, email, phone, contact_person, address, website, payment_terms) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Insert Products
    products_data = [
        (1, 'Premium Coffee Beans', 'Beverages', 25.99, 5, 20, 'QR001234', 'kg', 1, 'high', 15, 100),
        (2, 'Organic Green Tea', 'Beverages', 18.50, 12, 30, 'QR001235', 'kg', 2, 'medium', 25, 150),
        (3, 'Wireless Earbuds', 'Electronics', 89.99, 3, 15, 'QR001236', 'piece', 3, 'high', 10, 50),
        (4, 'Office Paper A4', 'Office Supplies', 4.99, 25, 50, 'QR001237', 'pack', 4, 'low', 40, 200),
        (5, 'Winter Gloves', 'Accessories', 12.99, 8, 25, 'QR001238', 'pair', 5, 'medium', 20, 100),
        (6, 'Smartphone Cases', 'Electronics', 15.99, 4, 20, 'QR001239', 'piece', 3, 'high', 15, 80),
        (7, 'Energy Drinks', 'Beverages', 2.99, 18, 50, 'QR001240', 'can', 6, 'medium', 40, 300),
        (8, 'USB Cables', 'Electronics', 8.99, 6, 25, 'QR001241', 'piece', 7, 'medium', 20, 100),
        (9, 'Notebook Set', 'Office Supplies', 6.99, 45, 30, 'QR001242', 'set', 4, 'low', 25, 150),
        (10, 'Hand Sanitizer', 'Health', 3.99, 2, 15, 'QR001243', 'bottle', 5, 'high', 12, 60)
    ]
    
    products_sql = """
    INSERT IGNORE INTO products (id, name, category, price, current_stock, low_stock_threshold, 
                                qr_code, unit, supplier_id, alert_priority, reorder_point, max_stock_level) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    try:
        with connection.cursor() as cursor:
            # Insert suppliers
            cursor.executemany(suppliers_sql, suppliers_data)
            print(f"✅ Inserted {cursor.rowcount} suppliers")
            
            # Insert products
            cursor.executemany(products_sql, products_data)
            print(f"✅ Inserted {cursor.rowcount} products")
            
            # Trigger alerts for low stock items
            cursor.execute("""
                UPDATE products 
                SET low_stock_alert_time = NOW() 
                WHERE current_stock <= low_stock_threshold
            """)
            print(f"✅ Triggered alerts for {cursor.rowcount} low stock items")
            
            connection.commit()
            
    except Exception as e:
        print(f"❌ Error inserting sample data: {e}")
        connection.rollback()

# Execute data insertion
if conn:
    insert_sample_data(conn)

In [ ]:
# Step 5: Verify Database Setup
def verify_database_setup(connection):
    """Verify all tables were created successfully and show status"""
    
    try:
        with connection.cursor() as cursor:
            # Check all tables exist
            cursor.execute("SHOW TABLES")
            tables = cursor.fetchall()
            
            print("📊 Database Tables Created:")
            print("-" * 40)
            for table in tables:
                table_name = list(table.values())[0]
                
                # Get row count
                cursor.execute(f"SELECT COUNT(*) as count FROM {table_name}")
                count = cursor.fetchone()['count']
                
                print(f"✅ {table_name:<20} | {count:>6} rows")
            
            print("\n🔍 Low Stock Alerts Summary:")
            print("-" * 40)
            
            # Check for low stock items
            cursor.execute("""
                SELECT 
                    COUNT(*) as total_products,
                    COUNT(CASE WHEN current_stock <= low_stock_threshold THEN 1 END) as low_stock_items,
                    COUNT(CASE WHEN current_stock = 0 THEN 1 END) as out_of_stock_items
                FROM products
            """)
            
            stats = cursor.fetchone()
            print(f"Total Products: {stats['total_products']}")
            print(f"Low Stock Items: {stats['low_stock_items']}")
            print(f"Out of Stock Items: {stats['out_of_stock_items']}")
            
            # Show current alerts
            cursor.execute("""
                SELECT p.name, p.current_stock, p.low_stock_threshold, p.alert_priority, s.name as supplier
                FROM products p
                LEFT JOIN suppliers s ON p.supplier_id = s.id
                WHERE p.current_stock <= p.low_stock_threshold
                ORDER BY p.alert_priority DESC, p.current_stock ASC
            """)
            
            alerts = cursor.fetchall()
            if alerts:
                print(f"\n🚨 Current Low Stock Alerts ({len(alerts)} items):")
                print("-" * 80)
                for alert in alerts:
                    print(f"• {alert['name']:<25} | Stock: {alert['current_stock']:>3}/{alert['low_stock_threshold']:<3} | "
                          f"Priority: {alert['alert_priority']:<6} | Supplier: {alert['supplier']}")
            
    except Exception as e:
        print(f"❌ Error verifying database: {e}")

# Run verification
if conn:
    verify_database_setup(conn)

## 2. Data Analysis & Visualization

Now that our database is properly set up, let's analyze the stock data and create visualizations to understand our inventory status.

In [ ]:
# Load and analyze all data
def load_and_analyze_data(connection):
    """Load data from database and create comprehensive analysis"""
    
    # Load products with supplier information
    products_query = """
    SELECT 
        p.*,
        s.name as supplier_name,
        s.email as supplier_email,
        s.payment_terms,
        CASE 
            WHEN p.current_stock = 0 THEN 'Out of Stock'
            WHEN p.current_stock <= (p.low_stock_threshold * 0.3) THEN 'Critical'
            WHEN p.current_stock <= (p.low_stock_threshold * 0.6) THEN 'Low'
            WHEN p.current_stock <= p.low_stock_threshold THEN 'Warning'
            ELSE 'Normal'
        END as stock_status,
        ROUND(((p.low_stock_threshold - p.current_stock) / p.low_stock_threshold) * 100, 2) as shortage_percentage
    FROM products p
    LEFT JOIN suppliers s ON p.supplier_id = s.id
    """
    
    # Load suppliers data
    suppliers_query = "SELECT * FROM suppliers"
    
    try:
        # Load data into DataFrames
        products_df = pd.read_sql(products_query, connection)
        suppliers_df = pd.read_sql(suppliers_query, connection)
        
        print("📊 Data Loaded Successfully!")
        print(f"Products: {len(products_df)} records")
        print(f"Suppliers: {len(suppliers_df)} records")
        
        return products_df, suppliers_df
        
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return None, None

# Load data
if conn:
    products_df, suppliers_df = load_and_analyze_data(conn)
    
    if products_df is not None:
        # Display basic statistics
        print("\n📈 Product Inventory Summary:")
        print("=" * 50)
        print(f"Total Products: {len(products_df)}")
        print(f"Average Stock Level: {products_df['current_stock'].mean():.1f}")
        print(f"Total Inventory Value: ${(products_df['current_stock'] * products_df['price']).sum():,.2f}")
        
        # Stock status distribution
        print(f"\n📊 Stock Status Distribution:")
        print("-" * 30)
        status_counts = products_df['stock_status'].value_counts()
        for status, count in status_counts.items():
            percentage = (count / len(products_df)) * 100
            print(f"{status:<12}: {count:>2} ({percentage:>5.1f}%)")

In [ ]:
# Create comprehensive visualizations
def create_visualizations(products_df, suppliers_df):
    """Create multiple charts for stock analysis"""
    
    # Set up the plotting area
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('AI Stock Management - Inventory Analysis Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Stock Status Distribution (Pie Chart)
    status_counts = products_df['stock_status'].value_counts()
    colors = ['#ff4444', '#ff8800', '#ffdd00', '#88dd00', '#00dd88']
    axes[0,0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', 
                  colors=colors[:len(status_counts)], startangle=90)
    axes[0,0].set_title('Stock Status Distribution')
    
    # 2. Current Stock vs Threshold (Scatter Plot)
    scatter = axes[0,1].scatter(products_df['low_stock_threshold'], products_df['current_stock'], 
                               c=products_df['price'], s=100, alpha=0.7, cmap='viridis')
    axes[0,1].plot([0, products_df['low_stock_threshold'].max()], [0, products_df['low_stock_threshold'].max()], 
                   'r--', alpha=0.5, label='Threshold Line')
    axes[0,1].set_xlabel('Low Stock Threshold')
    axes[0,1].set_ylabel('Current Stock')
    axes[0,1].set_title('Current Stock vs Threshold (Color = Price)')
    axes[0,1].legend()
    plt.colorbar(scatter, ax=axes[0,1], label='Price ($)')
    
    # 3. Products by Category (Bar Chart)
    category_counts = products_df['category'].value_counts()
    axes[0,2].bar(category_counts.index, category_counts.values, color='skyblue')
    axes[0,2].set_title('Products by Category')
    axes[0,2].set_xlabel('Category')
    axes[0,2].set_ylabel('Number of Products')
    axes[0,2].tick_params(axis='x', rotation=45)
    
    # 4. Alert Priority Distribution
    priority_counts = products_df['alert_priority'].value_counts()
    priority_colors = {'high': '#ff4444', 'medium': '#ffaa00', 'low': '#44aa44'}
    bars = axes[1,0].bar(priority_counts.index, priority_counts.values, 
                        color=[priority_colors[p] for p in priority_counts.index])
    axes[1,0].set_title('Alert Priority Distribution')
    axes[1,0].set_xlabel('Priority Level')
    axes[1,0].set_ylabel('Number of Products')
    
    # 5. Inventory Value by Category
    category_value = products_df.groupby('category').apply(lambda x: (x['current_stock'] * x['price']).sum()).sort_values(ascending=False)
    axes[1,1].bar(category_value.index, category_value.values, color='lightgreen')
    axes[1,1].set_title('Inventory Value by Category')
    axes[1,1].set_xlabel('Category')
    axes[1,1].set_ylabel('Total Value ($)')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    # 6. Supplier Performance (Products per Supplier)
    supplier_products = products_df['supplier_name'].value_counts()
    axes[1,2].barh(supplier_products.index, supplier_products.values, color='coral')
    axes[1,2].set_title('Products per Supplier')
    axes[1,2].set_xlabel('Number of Products')
    axes[1,2].set_ylabel('Supplier')
    
    plt.tight_layout()
    plt.show()
    
    # Additional detailed analysis
    print("\n🔍 Detailed Analysis:")
    print("=" * 50)
    
    # Low stock items
    low_stock = products_df[products_df['current_stock'] <= products_df['low_stock_threshold']]
    print(f"\n🚨 Low Stock Items ({len(low_stock)}):")
    if len(low_stock) > 0:
        print(low_stock[['name', 'current_stock', 'low_stock_threshold', 'alert_priority', 'supplier_name']].to_string(index=False))
    
    # High value items
    print(f"\n💰 Top 5 Most Valuable Items:")
    top_value = products_df.nlargest(5, 'price')[['name', 'price', 'current_stock', 'category']]
    print(top_value.to_string(index=False))
    
    # Supplier analysis
    print(f"\n🏢 Supplier Analysis:")
    supplier_analysis = products_df.groupby('supplier_name').agg({
        'current_stock': 'sum',
        'price': 'mean',
        'name': 'count'
    }).round(2)
    supplier_analysis.columns = ['Total_Stock', 'Avg_Price', 'Product_Count']
    print(supplier_analysis.to_string())

# Create visualizations if data is available
if conn and products_df is not None:
    create_visualizations(products_df, suppliers_df)

## 3. Corrected SQL Scripts for Manual Execution

### MySQL 8.0 Compatible Scripts

Here are the corrected SQL scripts that you can copy and paste directly into your MySQL command line. These scripts address all the syntax errors you encountered:

In [ ]:
-- ============================================================================
-- CORRECTED SQL SCRIPTS FOR MySQL 8.0.41
-- ============================================================================
-- Copy and paste these scripts into your MySQL command line

-- Step 1: Create Suppliers Table
CREATE TABLE IF NOT EXISTS suppliers (
  id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(255) NOT NULL,
  email VARCHAR(255),
  phone VARCHAR(20),
  address TEXT,
  contact_person VARCHAR(255),
  website VARCHAR(255),
  tax_number VARCHAR(50),
  payment_terms INT DEFAULT 30,
  is_active BOOLEAN DEFAULT TRUE,
  notes TEXT,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  INDEX idx_supplier_name (name),
  INDEX idx_supplier_active (is_active)
);

-- Step 2: Create Products Table
CREATE TABLE IF NOT EXISTS products (
  id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(255) NOT NULL,
  category VARCHAR(100),
  price DECIMAL(10,2) NOT NULL,
  current_stock INT DEFAULT 0,
  low_stock_threshold INT DEFAULT 10,
  low_stock_alert_time TIMESTAMP NULL,
  alert_status ENUM('active', 'ignored', 'resolved') DEFAULT 'active',
  qr_code VARCHAR(100) UNIQUE,
  unit VARCHAR(20) DEFAULT 'piece',
  supplier_id INT,
  alert_priority ENUM('low', 'medium', 'high') DEFAULT 'medium',
  reorder_point INT DEFAULT 0,
  max_stock_level INT DEFAULT 1000,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  FOREIGN KEY (supplier_id) REFERENCES suppliers(id) ON DELETE SET NULL,
  INDEX idx_products_stock (current_stock, low_stock_threshold),
  INDEX idx_products_category (category),
  INDEX idx_products_supplier (supplier_id)
);

-- Step 3: Create Purchase Orders Table
CREATE TABLE IF NOT EXISTS purchase_orders (
  id INT AUTO_INCREMENT PRIMARY KEY,
  po_number VARCHAR(50) UNIQUE NOT NULL,
  product_id INT NOT NULL,
  supplier_id INT,
  quantity INT NOT NULL,
  unit_price DECIMAL(10,2) NOT NULL,
  total_amount DECIMAL(10,2) NOT NULL,
  status ENUM('pending', 'sent', 'approved', 'delivered', 'completed', 'cancelled') DEFAULT 'pending',
  priority ENUM('low', 'medium', 'high', 'urgent') DEFAULT 'medium',
  notes TEXT,
  sent_method ENUM('email', 'whatsapp', 'phone', 'fax') NULL,
  sent_to VARCHAR(255) NULL,
  sent_at TIMESTAMP NULL,
  expected_delivery_date DATE NULL,
  actual_delivery_date DATE NULL,
  delivery_address TEXT,
  terms_and_conditions TEXT,
  created_by VARCHAR(255) DEFAULT 'System',
  approved_by VARCHAR(255) NULL,
  approved_at TIMESTAMP NULL,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE CASCADE,
  FOREIGN KEY (supplier_id) REFERENCES suppliers(id) ON DELETE SET NULL,
  INDEX idx_po_status (status),
  INDEX idx_po_date (created_at),
  INDEX idx_po_supplier (supplier_id),
  INDEX idx_po_product (product_id)
);

-- Step 4: Create Ignored Alerts Table
CREATE TABLE IF NOT EXISTS ignored_alerts (
  id INT AUTO_INCREMENT PRIMARY KEY,
  product_id INT NOT NULL,
  original_alert_time TIMESTAMP NOT NULL,
  reason TEXT,
  ignored_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  ignored_by VARCHAR(255) DEFAULT 'Admin',
  stock_level_when_ignored INT,
  can_reactivate BOOLEAN DEFAULT TRUE,
  reactivate_after TIMESTAMP NULL,
  FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE CASCADE,
  INDEX idx_ignored_product (product_id),
  INDEX idx_ignored_date (ignored_at)
);

-- Step 5: Create Alert History Table
CREATE TABLE IF NOT EXISTS alert_history (
  id INT AUTO_INCREMENT PRIMARY KEY,
  product_id INT NOT NULL,
  action ENUM('created', 'ignored', 'resolved', 'reactivated', 'po_generated') NOT NULL,
  stock_level INT,
  threshold_level INT,
  performed_by VARCHAR(255) DEFAULT 'System',
  notes TEXT,
  metadata JSON,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE CASCADE,
  INDEX idx_history_product (product_id),
  INDEX idx_history_action (action),
  INDEX idx_history_date (created_at)
);

In [ ]:
-- Step 6: Insert Sample Data

-- Insert Suppliers
INSERT IGNORE INTO suppliers (id, name, email, phone, contact_person, address, website, payment_terms) VALUES
(1, 'Coffee World Ltd', 'orders@coffeeworld.com', '+1-555-0101', 'John Smith', '123 Coffee St, Seattle, WA 98101', 'www.coffeeworld.com', 30),
(2, 'Herbal Supplies Co', 'supply@herbal.com', '+1-555-0102', 'Sarah Johnson', '456 Green Ave, Portland, OR 97201', 'www.herbalsupplies.com', 15),
(3, 'Tech Solutions Inc', 'orders@techsol.com', '+1-555-0103', 'Mike Chen', '789 Tech Blvd, San Jose, CA 95101', 'www.techsolutions.com', 45),
(4, 'Office Depot Pro', 'bulk@officedepot.com', '+1-555-0104', 'Lisa Brown', '321 Business Rd, Dallas, TX 75201', 'www.officedepot.com', 30),
(5, 'General Supplies Hub', 'info@generalsupplies.com', '+1-555-0105', 'David Wilson', '654 Supply Lane, Chicago, IL 60601', 'www.generalsupplies.com', 20),
(6, 'Fresh Foods Wholesale', 'wholesale@freshfoods.com', '+1-555-0106', 'Emma Davis', '987 Fresh Market St, Miami, FL 33101', 'www.freshfoods.com', 7),
(7, 'Electronic Components Ltd', 'sales@electronics.com', '+1-555-0107', 'James Miller', '147 Circuit Ave, Austin, TX 78701', 'www.electronics.com', 60);

-- Insert Products
INSERT IGNORE INTO products (id, name, category, price, current_stock, low_stock_threshold, qr_code, unit, supplier_id, alert_priority, reorder_point, max_stock_level) VALUES
(1, 'Premium Coffee Beans', 'Beverages', 25.99, 5, 20, 'QR001234', 'kg', 1, 'high', 15, 100),
(2, 'Organic Green Tea', 'Beverages', 18.50, 12, 30, 'QR001235', 'kg', 2, 'medium', 25, 150),
(3, 'Wireless Earbuds', 'Electronics', 89.99, 3, 15, 'QR001236', 'piece', 3, 'high', 10, 50),
(4, 'Office Paper A4', 'Office Supplies', 4.99, 25, 50, 'QR001237', 'pack', 4, 'low', 40, 200),
(5, 'Winter Gloves', 'Accessories', 12.99, 8, 25, 'QR001238', 'pair', 5, 'medium', 20, 100),
(6, 'Smartphone Cases', 'Electronics', 15.99, 4, 20, 'QR001239', 'piece', 3, 'high', 15, 80),
(7, 'Energy Drinks', 'Beverages', 2.99, 18, 50, 'QR001240', 'can', 6, 'medium', 40, 300),
(8, 'USB Cables', 'Electronics', 8.99, 6, 25, 'QR001241', 'piece', 7, 'medium', 20, 100),
(9, 'Notebook Set', 'Office Supplies', 6.99, 45, 30, 'QR001242', 'set', 4, 'low', 25, 150),
(10, 'Hand Sanitizer', 'Health', 3.99, 2, 15, 'QR001243', 'bottle', 5, 'high', 12, 60);

-- Trigger alerts for low stock items
UPDATE products SET low_stock_alert_time = NOW() WHERE current_stock <= low_stock_threshold;

-- Step 7: Create Indexes (MySQL 8.0 Compatible - without IF NOT EXISTS)
CREATE INDEX idx_products_low_stock ON products(current_stock, low_stock_threshold, alert_status);
CREATE INDEX idx_products_alert_time ON products(low_stock_alert_time);
CREATE INDEX idx_products_priority ON products(alert_priority);

-- Step 8: Verification Queries
-- Check all tables
SHOW TABLES;

-- View current stock status
SELECT 
  p.name,
  p.current_stock,
  p.low_stock_threshold,
  p.alert_priority,
  s.name as supplier_name,
  CASE 
    WHEN p.current_stock = 0 THEN 'Out of Stock'
    WHEN p.current_stock <= (p.low_stock_threshold * 0.3) THEN 'Critical'
    WHEN p.current_stock <= (p.low_stock_threshold * 0.6) THEN 'Low'
    WHEN p.current_stock <= p.low_stock_threshold THEN 'Warning'
    ELSE 'Normal'
  END as stock_status
FROM products p
LEFT JOIN suppliers s ON p.supplier_id = s.id
WHERE p.current_stock <= p.low_stock_threshold
ORDER BY p.alert_priority DESC, p.current_stock ASC;

## 4. Troubleshooting Guide

### Common MySQL 8.0 Issues & Solutions

#### 1. **Syntax Error: `IF NOT EXISTS` in ALTER TABLE**
❌ **Error**: `You have an error in your SQL syntax... near 'IF NOT EXISTS low_stock_threshold'`

✅ **Solution**: Remove `IF NOT EXISTS` from `ALTER TABLE` statements. Use separate `ALTER TABLE` statements for each column.

#### 2. **Table 'products' doesn't exist**
❌ **Error**: `Table 'demodemo.products' doesn't exist`

✅ **Solution**: Create tables in the correct dependency order:
1. `suppliers` (no dependencies)
2. `products` (depends on suppliers)
3. `purchase_orders` (depends on products and suppliers)
4. `ignored_alerts` (depends on products)

#### 3. **Index Creation Syntax Error**
❌ **Error**: `You have an error in your SQL syntax... near 'IF NOT EXISTS idx_products_low_stock'`

✅ **Solution**: MySQL 8.0 doesn't support `IF NOT EXISTS` in `CREATE INDEX`. Use plain `CREATE INDEX` statements.

#### 4. **Foreign Key Constraints**
❌ **Error**: `Failed to open the referenced table 'products'`

✅ **Solution**: Ensure parent tables exist before creating child tables with foreign keys.

### Database Setup Checklist

✅ **Step 1**: Create `suppliers` table  
✅ **Step 2**: Create `products` table with foreign key to suppliers  
✅ **Step 3**: Create dependent tables (`purchase_orders`, `ignored_alerts`, `alert_history`)  
✅ **Step 4**: Insert sample data  
✅ **Step 5**: Create indexes  
✅ **Step 6**: Verify setup with test queries  

### Quick Commands for Testing

```sql
-- Check if all tables exist
SHOW TABLES;

-- Count records in each table
SELECT 'suppliers' as table_name, COUNT(*) as records FROM suppliers
UNION ALL
SELECT 'products', COUNT(*) FROM products
UNION ALL
SELECT 'purchase_orders', COUNT(*) FROM purchase_orders
UNION ALL
SELECT 'ignored_alerts', COUNT(*) FROM ignored_alerts
UNION ALL
SELECT 'alert_history', COUNT(*) FROM alert_history;

-- View current low stock alerts
SELECT name, current_stock, low_stock_threshold, alert_priority 
FROM products 
WHERE current_stock <= low_stock_threshold;
```

## 5. Summary & Next Steps

### 🎯 What We've Accomplished

1. **✅ Fixed MySQL 8.0 Compatibility Issues**
   - Removed unsupported `IF NOT EXISTS` syntax from `ALTER TABLE` and `CREATE INDEX`
   - Created proper table dependency order
   - Fixed foreign key constraint issues

2. **✅ Complete Database Setup**
   - Created all required tables for Low Stock Alerts system
   - Inserted comprehensive sample data for testing
   - Added proper indexes for performance

3. **✅ Data Analysis & Visualization**
   - Created comprehensive inventory analysis dashboard
   - Generated stock status reports and visualizations
   - Provided supplier performance metrics

### 🚀 Next Steps for Your Application

1. **Backend Integration**
   - Use the corrected SQL scripts in your Node.js backend
   - Update your API endpoints to match the new table structure
   - Test all CRUD operations with the new schema

2. **Frontend Updates**
   - Verify the React Low Stock Alerts page works with new data
   - Test the purchase order generation workflow
   - Confirm supplier communication features

3. **Database Optimization**
   - Monitor query performance with the new indexes
   - Consider adding more indexes based on usage patterns
   - Set up regular maintenance tasks for optimal performance

### 📋 Quick Setup Commands

Copy and paste these commands in your MySQL command line:

```bash
# Connect to MySQL
mysql -u root -p

# Use your database
USE DemoDemo;

# Then copy and paste the SQL scripts from cell 4 and 5 above
```

### 🔗 Integration with Your React App

Your Low Stock Alerts page should now work perfectly with this database structure. The key endpoints your backend should provide:

- `GET /api/alerts/low-stock` - Get all low stock alerts
- `POST /api/alerts/ignore` - Ignore an alert
- `POST /api/alerts/purchase-order` - Create purchase order
- `GET /api/alerts/ignored` - Get ignored alerts
- `GET /api/alerts/resolved` - Get resolved alerts

The database is now ready for your AI Stock Management system! 🎉